In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
# setting Jedha color palette as default
pio.templates["jedha"] = go.layout.Template(
    layout_colorway=["#4B9AC7", "#4BE8E0", "#9DD4F3", "#97FBF6", "#2A7FAF", "#23B1AB", "#0E3449", "#015955"]
)
pio.templates.default = "jedha"
pio.renderers.default = "iframe" # to be replaced by "iframe" if working on JULIE

In [2]:
dataset = pd.read_csv("walmart_store_sales.csv")

In [3]:
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [4]:
# Basic stats
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

print("Basics statistics: ")
data_desc = dataset.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Number of rows : 150
Number of columns : 8

Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

In [5]:
#Drop null values in Weekly_Sales column

dataset = dataset[dataset['Weekly_Sales'].notna()]
print("The following will be 'False' if there's no missing values in the dataset: ", dataset['Weekly_Sales'].isnull().any().any())

The following will be 'False' if there's no missing values in the dataset:  False


In [6]:
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))
print()

Number of rows : 136
Number of columns : 8



In [7]:
#Drop null values in date columns
dataset = dataset[dataset['Date'].notna()]
print("Number of rows : {}".format(dataset.shape[0]))
print("Number of columns : {}".format(dataset.shape[1]))

Number of rows : 118
Number of columns : 8


In [8]:
print("Percentage of missing values: ")
display(100*dataset.isnull().sum()/dataset.shape[0])

Percentage of missing values: 


Store           0.000000
Date            0.000000
Weekly_Sales    0.000000
Holiday_Flag    7.627119
Temperature     9.322034
Fuel_Price      9.322034
CPI             7.627119
Unemployment    9.322034
dtype: float64

In [9]:
dataset.dtypes

Store           float64
Date             object
Weekly_Sales    float64
Holiday_Flag    float64
Temperature     float64
Fuel_Price      float64
CPI             float64
Unemployment    float64
dtype: object

In [10]:
#convert date column to datetype
dataset['Date']= pd.to_datetime(dataset['Date'],format='%d-%m-%Y')
dataset.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,2011-02-18,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,2011-03-25,1807545.43,0.0,42.38,3.435,128.616064,7.470
4,6.0,2010-05-28,1644470.66,0.0,78.89,2.759,212.412888,7.092
5,4.0,2010-05-28,1857533.70,0.0,NaN,2.756,126.160226,7.896
6,15.0,2011-06-03,695396.19,0.0,69.80,4.069,134.855161,7.658


In [11]:
# split Date column into 3 parts (year, month, day and day of week) and drop Date column
dataset['DayOfWeek'] =dataset['Date'].dt.dayofweek
dataset['Day'] =dataset['Date'].dt.day
dataset['Month'] = dataset['Date'].dt.month
dataset['Year'] = dataset['Date'].dt.year
dataset = dataset.drop(columns=['Date'])
dataset.reset_index(drop=True)
dataset.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,DayOfWeek,Day,Month,Year
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,4,18,2,2011
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,4,25,3,2011
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,4,28,5,2010
5,4.0,1857533.70,0.0,NaN,2.756,126.160226,7.896,4,28,5,2010
6,15.0,695396.19,0.0,69.80,4.069,134.855161,7.658,4,3,6,2011


In [12]:
#Drop lines containing invalid values or outliers : 
def remove_outliers():
    numerical_col_names = ['Temperature', 'Fuel_Price', 'Fuel_Price', 'CPI', 'Unemployment']
    for col in numerical_col_names:
        mean = np.mean(dataset[col])
        sigma = np.std(dataset[col])
        print("Mean: {:.2f}".format(mean))
        print("Standard Deviation: {:.2f}".format(sigma))
        lower_range = mean-(3*sigma)
        upper_range = mean+(3*sigma)
        outliers = [i for i in dataset[col] if i<lower_range or i>upper_range]
        print("Number of outliers:",len(outliers))
        dataset.drop(dataset[(dataset[col]<lower_range) | (dataset[col]>upper_range)].index, inplace=True)
    print(dataset.shape)   

In [13]:
remove_outliers()

Mean: 60.71
Standard Deviation: 17.88
Number of outliers: 0
Mean: 3.29
Standard Deviation: 0.48
Number of outliers: 0
Mean: 3.29
Standard Deviation: 0.48
Number of outliers: 0
Mean: 177.72
Standard Deviation: 39.63
Number of outliers: 0
Mean: 7.68
Standard Deviation: 1.67
Number of outliers: 5
(113, 11)


In [14]:
display(dataset.isnull())

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,DayOfWeek,Day,Month,Year
0,False,False,True,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False
5,False,False,False,True,False,False,False,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
144,False,False,False,False,False,False,False,False,False,False,False
145,False,False,False,False,False,False,False,False,False,False,False
147,False,False,False,False,False,False,True,False,False,False,False
148,False,False,False,False,False,False,True,False,False,False,False


In [15]:
dataset.describe()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,DayOfWeek,Day,Month,Year
count,113.000000,1.130000e+02,104.000000,103.000000,102.000000,104.000000,102.000000,113.0,113.000000,113.000000,113.000000
mean,9.858407,1.267415e+06,0.067308,60.197087,3.273863,180.105389,7.376775,4.0,16.530973,6.274336,2010.831858
std,6.184467,6.746824e+05,0.251767,17.878511,0.481421,39.201866,0.973078,0.0,8.238705,3.179869,0.822699
min,1.000000,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000,4.0,1.000000,1.000000,2010.000000
25%,4.000000,5.634608e+05,0.000000,45.020000,2.814750,132.579257,6.642250,4.0,10.000000,4.000000,2010.000000
50%,9.000000,1.420405e+06,0.000000,61.110000,3.302500,197.500965,7.404500,4.0,17.000000,6.000000,2011.000000
75%,15.000000,1.847431e+06,0.000000,75.255000,3.683500,214.809008,8.096750,4.0,24.000000,9.000000,2012.000000
max,20.000000,2.771397e+06,1.000000,91.650000,4.170000,226.968844,9.524000,4.0,31.000000,12.000000,2012.000000


In [16]:
# Separate target variable Y from features X
print("Separating labels from features...")
target_variable = "Weekly_Sales"

X = dataset.drop(target_variable, axis = 1)
Y = dataset.loc[:,target_variable]

print("...Done.")
print()

print('Y : ')
print(Y.head())
print()
print('X :')
X.head()

Separating labels from features...
...Done.

Y : 
0    1572117.54
1    1807545.43
4    1644470.66
5    1857533.70
6     695396.19
Name: Weekly_Sales, dtype: float64

X :


,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,DayOfWeek,Day,Month,Year
0,6.0,NaN,59.61,3.045,214.777523,6.858,4,18,2,2011
1,13.0,0.0,42.38,3.435,128.616064,7.470,4,25,3,2011
4,6.0,0.0,78.89,2.759,212.412888,7.092,4,28,5,2010
5,4.0,0.0,NaN,2.756,126.160226,7.896,4,28,5,2010
6,15.0,0.0,69.80,4.069,134.855161,7.658,4,3,6,2011


In [17]:
categorical_features = ['Store', 'Holiday_Flag'] # Names of categorical columns in X_train/X_test

In [18]:
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'DayOfWeek'] # Names of numeric columns in X_train/X_test


In [19]:
from sklearn.model_selection import train_test_split
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15, random_state=0)
print("...Done.")
print()

Dividing into train and test sets...
...Done.



In [20]:
#Preprocessings to be planned with scikit-learn
#Categorical variables : Store, Holiday_Flag
from sklearn.preprocessing import OneHotEncoder

categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(drop='first'))# missing values will be replaced by most frequent value
    ])

In [21]:
#Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, DayOfWeek
# Create pipeline for numeric features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')), # missing values in Age will be replaced by columns' mean
    ('scaler', StandardScaler())
])

In [22]:
from sklearn.compose import ColumnTransformer

In [23]:
# Use ColumnTranformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5,:])
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head())
X_test = preprocessor.transform(X_test) 
print('...Done.')
print(X_test[0:5,:])
print(X_test)

Performing preprocessings on train set...
     Store  Holiday_Flag  Temperature  Fuel_Price         CPI  Unemployment  \
21     5.0           0.0        77.38       3.899  216.534361         6.489   
16    13.0           0.0        76.34       2.850         NaN         7.951   
27    16.0           0.0        43.95       3.828  192.831317         6.339   
112    2.0           0.0        39.69       2.514  210.945160           NaN   
118    9.0           NaN        82.99       2.637  215.016648         6.384   

     DayOfWeek  Day  Month  Year  
21           4   13      5  2011  
16           4   20      8  2010  
27           4   20      5  2011  
112          4   19      2  2010  
118          4   18      6  2010  
...Done.
[[ 1.03506392e+00  1.33284088e+00  1.01407867e+00 -9.57222356e-01
   2.08514414e-01 -3.30121111e-01 -4.26404974e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00


In [24]:
# Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

Train model...
...Done.


In [25]:
# Print R^2 scores
print("R2 score on training set : ", regressor.score(X_train, Y_train))
print("R2 score on test set : ", regressor.score(X_test, Y_test))

R2 score on training set :  0.9737690376310432
R2 score on test set :  0.932200319474471


In [26]:
#get columns_names of each coefficient
column_names = []
for name, step, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = step.named_steps['encoder'].get_feature_names_out(categorical_features) # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)


Names of columns corresponding to each coefficient:  ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year', 'Month', 'Day', 'DayOfWeek', 'Store_2.0', 'Store_3.0', 'Store_4.0', 'Store_5.0', 'Store_6.0', 'Store_7.0', 'Store_8.0', 'Store_9.0', 'Store_10.0', 'Store_11.0', 'Store_13.0', 'Store_14.0', 'Store_15.0', 'Store_16.0', 'Store_17.0', 'Store_18.0', 'Store_19.0', 'Store_20.0', 'Holiday_Flag_1.0']


In [27]:
#get coefficients and put into a dataframe
coefs = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs

,coefficients
Temperature,-3.361042e+04
Fuel_Price,-3.924436e+04
CPI,-3.900737e+04
Unemployment,-7.329573e+04
Year,-6.999225e+03
Month,7.001737e+04
Day,-3.750099e+04
DayOfWeek,3.492460e-10
Store_2.0,3.984815e+05
Store_3.0,-1.195880e+06


In [28]:
# Plot feature_importance
feature_importance = abs(coefs).sort_values(by = 'coefficients')
# Plot coefficients
fig = px.bar(feature_importance, orientation = 'h')
fig.update_layout(showlegend = False, 
                  margin = {'l': 120} # to avoid cropping of column names
                 )
fig.show()

In [29]:
## The most important feature is Temperature

In [30]:
##Let's focus on Ridge regularization. We'll train 3 Ridge regressors with different values of the strength alpha, and analyze the performances as well as the influence on the model's coefficients.

In [31]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Ridge model
#print("3-fold cross-validation...")
#regressor = Ridge()
#scores = cross_val_score(regressor, X_train, Y_train, cv=3)
#print('The cross-validated R2-score is : ', scores.mean())
#print('The standard deviation is : ', scores.std())

In [32]:
ridge1 = Ridge()
print(ridge1)
ridge1.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", ridge1.score(X_train, Y_train))
print("R2 score on test set : ", ridge1.score(X_test, Y_test))

Ridge()
R2 score on training set :  0.9346260591996387
R2 score on test set :  0.9130236781156538


In [33]:
ridge2 = Ridge(alpha = 20)
print(ridge2)
ridge2.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", ridge2.score(X_train, Y_train))
print("R2 score on test set : ", ridge2.score(X_test, Y_test))

Ridge(alpha=20)
R2 score on training set :  0.4378183089117005
R2 score on test set :  0.4710368838594654


In [34]:
ridge3 = Ridge(alpha = 50)
print(ridge3)
ridge3.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", ridge3.score(X_train, Y_train))
print("R2 score on test set : ", ridge3.score(X_test, Y_test))

Ridge(alpha=50)
R2 score on training set :  0.2730489683067082
R2 score on test set :  0.31727046196348996


In [35]:
data_dict = {
    'Feature': column_names,
    'Ridge1': ridge1.coef_,
    'Ridge2': ridge2.coef_,
    'Ridge3': ridge3.coef_
            }

coefficients_ridge = pd.DataFrame(data = data_dict)
coefficients_ridge.head()

,Feature,Ridge1,Ridge2,Ridge3
0,Temperature,-32203.377338,-65153.244281,-60149.075223
1,Fuel_Price,-49484.402068,-23469.192266,-12639.052229
2,CPI,-69670.253175,-137233.301294,-117490.715171
3,Unemployment,26285.817091,56379.259526,48004.348346
4,Year,26617.421162,-9393.599174,-16425.318862


In [36]:
fig = px.line(coefficients_ridge, x = 'Feature', y = ['Ridge1', 'Ridge2', 'Ridge3'])
fig.show()

In [37]:
##With Ridge regularization, when alpha increases, all the values of the coefficients decrease.

In [38]:
# Perform 3-fold cross-validation to evaluate the generalized R2 score obtained with a Lasso model
print("3-fold cross-validation...")
regressor = Lasso()
scores = cross_val_score(regressor, X_train, Y_train, cv=3)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())

3-fold cross-validation...
The cross-validated R2-score is :  0.8703210267449557
The standard deviation is :  0.0666365035227831


In [39]:
##Let's make the same study with Lasso regularization.
lasso1 = Lasso(alpha = 1)
print(lasso1)
lasso1.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", lasso1.score(X_train, Y_train))
print("R2 score on test set : ", lasso1.score(X_test, Y_test))

Lasso(alpha=1)
R2 score on training set :  0.9737690359306964
R2 score on test set :  0.9322140987329915


In [40]:
lasso2 = Lasso(alpha = 20)
print(lasso2)
lasso2.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", lasso2.score(X_train, Y_train))
print("R2 score on test set : ", lasso2.score(X_test, Y_test))

Lasso(alpha=20)
R2 score on training set :  0.9737683296795752
R2 score on test set :  0.9325010204212417


In [41]:
lasso3 = Lasso(alpha = 50)
print(lasso3)
lasso3.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", lasso3.score(X_train, Y_train))
print("R2 score on test set : ", lasso3.score(X_test, Y_test))

Lasso(alpha=50)
R2 score on training set :  0.9737644087898027
R2 score on test set :  0.9330100677648602


In [42]:
data_dict = {
    'Feature': column_names,
    'Lasso1': lasso1.coef_,
    'Lasso2': lasso2.coef_,
    'Lasso3': lasso3.coef_
            }

coefficients_lasso = pd.DataFrame(data = data_dict)
coefficients_lasso.head()

,Feature,Lasso1,Lasso2,Lasso3
0,Temperature,-33601.663557,-33440.924802,-33200.572576
1,Fuel_Price,-39246.333959,-39278.826416,-39318.223249
2,CPI,-38984.595954,-38394.029271,-37089.634815
3,Unemployment,-73282.950470,-73040.055894,-72656.138413
4,Year,-6992.587110,-6877.945545,-6724.008698


In [43]:
fig = px.line(coefficients_lasso, x = 'Feature', y = ['Lasso1', 'Lasso2', 'Lasso3'])
fig.show()

In [44]:
#Contrary to Ridge, Lasso regularization assigns many of the model's coefficients exactly to zero. The higher alpha is, the more coefficients are cancelled in the model.

In [47]:
#The regularization strength

# Perform grid search
print("Grid search...")
regressor = Ridge()
# Grid of values to be tested
params = {
    'alpha': [1, 2, 3,10, 20, 30,40, 50]
}
best_ridge = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
best_ridge.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", best_ridge.best_params_)
print("Best R2 score : ", best_ridge.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 1}
Best R2 score :  0.8245263578681762


In [48]:
# Perform grid search
print("Grid search...")
regressor = Lasso()
# Grid of values to be tested
params = {
    'alpha': [1, 2, 3, 5, 10, 20, 30, 40]
}
best_lasso = GridSearchCV(regressor, param_grid = params, cv = 3) # cv : the number of folds to be used for CV
best_lasso.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", best_lasso.best_params_)
print("Best R2 score : ", best_lasso.best_score_)

Grid search...
...Done.
Best hyperparameters :  {'alpha': 40}
Best R2 score :  0.8710790263963711
